# Training Classifier

In [37]:
! rm -rf /content/Gren

In [38]:
! git clone https://github.com/Diwakar-Gupta/Gren.git

Cloning into 'Gren'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 154 (delta 20), reused 150 (delta 16), pack-reused 0
Receiving objects: 100% (154/154), 1.26 MiB | 16.93 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [39]:
import tensorflow as tf

In [40]:
DATASET_PATH = '/content/Gren/DataSet'
SEED = 90
VALIDATION_SPLIT = 0.2
IMAGE_SIZE=(100, 100)

In [63]:
train_data = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=SEED,
    subset='training',
    validation_split=VALIDATION_SPLIT,
    crop_to_aspect_ratio=False,
)

Found 82 files belonging to 7 classes.
Using 66 files for training.


In [64]:
validation_data = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=SEED,
    subset='validation',
    validation_split=VALIDATION_SPLIT,
    crop_to_aspect_ratio=False,
)

Found 82 files belonging to 7 classes.
Using 16 files for validation.


In [65]:
print(train_data.class_names)

TOTAL_CLASSES = len(train_data.class_names)

def get_class_name(i):
    return train_data.class_names[i]

['Beet_Sugar', 'Cane_Sugar', 'Olive_Oil', 'Palm_Oil', 'Rapeseed_Oil', 'Soyabeen_Oil', 'Sunflower_Oil']


In [ ]:
def preprocess(image, label):
    image = tf.image.resize(image, IMAGE_SIZE)
    # image = tf.image.random_crop(image, size=[*IMAGE_SIZE, 3])
    # image = tf.image.random_flip_left_right(image)
    # image = tf.keras.preprocessing.image.random_rotation(image.numpy(), 45,)
    # image = tf.image.random_brightness(image, 0.2)
    # image = tf.image.random_flip_up_down(image)
    image = tf.keras.applications.xception.preprocess_input(image)
    return image, label

train_data = train_data.map(preprocess)
validation_data = validation_data.map(preprocess)

In [57]:
base_model = tf.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
)

# base_model.summary()

In [58]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(TOTAL_CLASSES, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.inputs, outputs=output)

# model.summary()

In [59]:
model.compile(
    optimizer='Nadam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [60]:
base_model.trainable = False

model.fit(train_data, validation_data=validation_data, epochs=6,)

Epoch 1/6
3/3 [==============================] - 36s 7s/step - loss: 1.8102 - accuracy: 0.3049 - val_loss: 1.5216 - val_accuracy: 0.5854
Epoch 2/6
3/3 [==============================] - 21s 6s/step - loss: 1.3277 - accuracy: 0.7927 - val_loss: 1.6899 - val_accuracy: 0.5366
Epoch 3/6
3/3 [==============================] - 20s 6s/step - loss: 0.7801 - accuracy: 0.8415 - val_loss: 1.0981 - val_accuracy: 0.6341
Epoch 4/6
3/3 [==============================] - 19s 6s/step - loss: 0.3837 - accuracy: 0.9146 - val_loss: 0.7511 - val_accuracy: 0.7317
Epoch 5/6
3/3 [==============================] - 23s 8s/step - loss: 0.2879 - accuracy: 0.8780 - val_loss: 0.5113 - val_accuracy: 0.8171
Epoch 6/6
3/3 [==============================] - 19s 6s/step - loss: 0.1952 - accuracy: 0.9268 - val_loss: 0.2905 - val_accuracy: 0.8780


In [61]:
base_model.trainable = True

nadam = tf.keras.optimizers.Nadam(learning_rate=0.0001,)

model.compile(
    optimizer=nadam,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

model.fit(train_data, validation_data=validation_data, epochs=2,)

Epoch 1/2
3/3 [==============================] - 35s 7s/step - loss: 0.0350 - accuracy: 1.0000 - val_loss: 0.2441 - val_accuracy: 0.9268
Epoch 2/2
3/3 [==============================] - 26s 9s/step - loss: 0.0415 - accuracy: 1.0000 - val_loss: 0.1378 - val_accuracy: 0.9390


In [50]:
model.save('model')

INFO:tensorflow:Assets written to: model/assets


In [67]:
! zip -r model.zip model

  adding: model/ (stored 0%)
  adding: model/assets/ (stored 0%)
  adding: model/keras_metadata.pb (deflated 95%)
  adding: model/variables/ (stored 0%)
  adding: model/variables/variables.index (deflated 81%)
  adding: model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: model/saved_model.pb (deflated 92%)


In [68]:
from google.colab import files
files.download('model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>